In [ ]:
!pip install tensorflow-gpu==2.3.0

In [1]:
import os, h5py
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import load_model

from track import SOLAR_LOGG, SOLAR_TEFF, SOLAR_ZX

print('tensorflow ==', tf.__version__)

%matplotlib inline

tensorflow == 2.3.0


In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]

In [ ]:
# tf.device('XLA_GPU')

In [ ]:
name = 'model_2'

In [ ]:
path = '/fast/extended_grid.h5'
h5f = h5py.File(path, 'r')
data = np.array(h5f['data'])
columns = np.array([v.decode() for v in h5f['label']])
h5f.close()

In [ ]:
mask = [True, False, True, True, True, True, False, True, True, True, False, False, False, False]

In [ ]:
df = pd.DataFrame(data[:,mask], columns=columns[mask])
# del data

In [ ]:
columns

In [ ]:
# df.head()

In [ ]:
df['log(R/Rsun)'] = np.log10(data[:,9])
df['log(Age)(Gyr)'] = np.log10(data[:,1])
df['log(Prot)(days)'] = np.log10(data[:,6])
df['[Fe/H](surface)'] = np.log10(data[:,11]) - np.log10(SOLAR_ZX)
df['log(L/Lsun)'] = 2 * np.log10(data[:,9]) + 4 * (data[:,7] - np.log10(SOLAR_TEFF))

In [ ]:
df.head()

In [ ]:
mask = data[:,1] == 0
del data

In [ ]:
df_ = df.drop(df.loc[mask].index)
del df

In [ ]:
# df.drop(columns=['rho(g/cm3)','Xcore', 'Lnuc/Lsun'], inplace=True)

In [ ]:
# df['log(R/Rsun)'] = np.log10(df['R/Rsun'])
# df.drop(columns=['R/Rsun'], inplace=True)

In [ ]:
# df['log(g)'] = SOLAR_LOGG + np.log10(df['M/Msun']) - 2 * df['log(R/Rsun)']
# df['log(L/Lsun)'] = 2 * df['log(R/Rsun)'] + 4 * (df['log(Teff)(K)'] - np.log10(SOLAR_TEFF))
# df['[Fe/H](surface)'] = np.log10(df['Z/X(surface)']) - np.log10(SOLAR_ZX)
# df.drop(columns=['Z/X(surface)'], inplace=True)

In [ ]:
# df['log(Prot)(days)'] = np.log10(df['Prot(days)'])
# df.drop(columns=['Prot(days)'], inplace=True)

In [ ]:
# df['log(Age)(Gyr)'] = np.log10(df['Age(Gyr)'])
# df.drop(columns=['Age(Gyr)'], inplace=True)

In [ ]:
# non_diff = df.loc[df['EEP'] == 1, '[Fe/H](surface)'].to_numpy() == df.loc[df['EEP'] == 2, '[Fe/H](surface)'].to_numpy()
# np.sum(non_diff)

In [ ]:
# start = df.loc[df['EEP'] == 1, '[Fe/H](surface)'].index.to_numpy()
# stop = df.loc[df['EEP'] == 3, '[Fe/H](surface)'].index.to_numpy()

In [ ]:
# non_diff_idx = np.concatenate([np.arange(st, sp+1, dtype=int) for st, sp in zip(start[non_diff], stop[non_diff])])

In [ ]:
# len(non_diff_idx) / len(df)

In [ ]:
# len(df) - len(non_diff_idx)

In [ ]:
# df.drop(non_diff_idx, inplace=True)

In [ ]:
# len(df)

In [ ]:
# df2 = df.drop(columns=['R/Rsun', 'Xcore', 'Lnuc/Lsun', 'Z/X(surface)', 'L/Lsun', 'Prot(days)'])
# del df

In [ ]:
# df2.head()

In [ ]:
random_state = 2

train = df_.sample(frac=0.8, random_state=random_state)
test = df_.drop(train.index)

In [ ]:
print('Train length = ', len(train))
print('Test length = ', len(test))

In [ ]:
del df_

In [ ]:
input_cols = ['log(Age)(Gyr)', 'M/Msun', '[Fe/H]', 'alphaMLT', 'fk', 'Rocrit']
output_cols = ['log(Teff)(K)', 'log(R/Rsun)', '[Fe/H](surface)', 'log(Prot)(days)']

In [ ]:
sns.pairplot(train[input_cols].sample(10000), diag_kind='kde');

In [ ]:
sns.pairplot(train[output_cols].sample(10000), diag_kind='kde');

In [ ]:
input_normalizer = preprocessing.Normalization()
input_normalizer.adapt(train[input_cols].to_numpy())

In [ ]:
input_mean = input_normalizer.mean.numpy()
input_var = input_normalizer.variance.numpy()
for col, mean, var in zip(input_cols, input_mean, input_var):
    print(f'{col:<15}: {mean:.3e}, {var:.3e}')

In [ ]:
output_normalizer = preprocessing.Normalization()
output_normalizer.adapt(train[output_cols].to_numpy())

In [ ]:
output_mean = output_normalizer.mean.numpy()
output_var = output_normalizer.variance.numpy()
for col, mean, var in zip(output_cols, output_mean, output_var):
    print(f'{col:<15}: {mean:.3e}, {var:.3e}')

In [ ]:
output_rescaler = preprocessing.Rescaling(scale=np.sqrt(output_var))#, 
                                          # offset=output_mean)

In [ ]:
def build_model(n_neurons, n_hidden, activation='elu', l2_reg=0.0):
    hidden_layers = [layers.Dense(n_neurons, 
                                  activation=activation) for layer in range(n_hidden)]

    all_layers = [input_normalizer] + hidden_layers + [layers.Dense(len(output_cols)), output_rescaler]
    model = keras.Sequential(all_layers)
    return model

In [ ]:
# def build_model(n_neurons, n_hidden, activation='elu', l2_reg=0.0):
#     reg = regularizers.L2(l2_reg)
#     hidden_layers = [layers.Dense(n_neurons, 
#                                   activation=activation, 
#                                   kernel_regularizer=reg) for layer in range(n_hidden)]
#     all_layers = [input_normalizer] + hidden_layers + [layers.Dense(len(output_cols)), output_rescaler]
#     model = keras.Sequential(all_layers)
#     return model

In [ ]:
model = build_model(128, 6)#, l2_reg=0.0)

In [ ]:
model.save(f'models/{name}')

In [ ]:
learning_rate = 1e-4
momentum = 0.999

model.compile(loss='mean_squared_error',
#              optimizer=keras.optimizers.SGD(learning_rate, momentum=momentum),
             optimizer=keras.optimizers.Adam(learning_rate)
             )

In [ ]:
model.summary()

In [ ]:
batch_size = 8191*2

print('Number of batches:', len(train)//batch_size)
print('Remainder:', len(train)%batch_size)

In [ ]:
%%time
history = model.fit(
    train[input_cols], train[output_cols],
    validation_split=0.2,
    batch_size=batch_size,
    verbose=0, epochs=50,
)

In [ ]:
sns.lineplot(x=history.epoch, y=history.history['loss'], label='train loss')
sns.lineplot(x=history.epoch, y=history.history['val_loss'], label='validation loss')

plt.yscale('log')
plt.legend()

In [ ]:
model.save(f'models/{name}')

In [ ]:
pred = pd.DataFrame(model.predict(test[input_cols], batch_size=batch_size), columns=output_cols)
pred.head()

In [ ]:
truth = test[output_cols].reset_index(drop=True)
error = truth - pred
frac_error = (truth - pred) / truth

In [ ]:
truth.head()

In [ ]:
log_cols = ['log(Teff)(K)', 'log(R/Rsun)', 'log(Prot)(days)']

for col in log_cols:
    error[f'10^{col}'] = 10**truth[col] - 10**pred[col]
    frac_error[f'10^{col}'] = (10**truth[col] - 10**pred[col]) / 10**truth[col]

In [ ]:
error.head()

In [ ]:
error.mean()

In [ ]:
error.std()

In [ ]:
frac_error.mean()

In [ ]:
frac_error.std()

In [ ]:
from scipy import stats

In [ ]:
pd.Series(stats.median_abs_deviation(error), error.columns)

In [ ]:
pd.Series(stats.median_abs_deviation(frac_error), frac_error.columns)

In [ ]:
plt.plot(test['EEP'], 10**error['log(Prot)(days)'], '.');

In [ ]:
plt.plot(10**truth['log(Prot)(days)'], 10**error['log(Prot)(days)'], '.');

In [ ]:
plt.plot(10**truth['log(R/Rsun)'], 10**error['log(R/Rsun)'], '.');

In [ ]:
plt.plot(10**truth['log(Teff)(K)'], 10**error['log(Teff)(K)'], '.');

In [ ]:
ax = test.iloc[:5000].plot(x='log(Teff)(K)', y='log(L/Lsun)', c='M/Msun', cmap='viridis', kind='scatter')

ax.invert_xaxis();